# connect to Drive

In [1]:
#  connect to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install & import

In [2]:
!pip install torch==2.2.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.3 MB/s eta 0:00:00
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [3]:
!python -c "import torch; print(torch.__version__)"


2.2.1+cu121


In [4]:
!pip install pyg-lib -f https://data.pyg.org/whl/torch-2.2.1+cu121.html
!pip install torch-geometric
!pip install faiss-gpu

Looking in links: https://data.pyg.org/whl/torch-2.2.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.4 MB/s eta 0:00:00


In [5]:
import pandas as pd

import argparse
import os.path as osp

import torch
import torch.nn.functional as F
from tqdm import tqdm

import torch_geometric.transforms as T
from torch_geometric import EdgeIndex
from torch_geometric.loader import LinkNeighborLoader, NeighborLoader
from torch_geometric.data import NeighborSampler
from torch_geometric.metrics import (
    LinkPredPrecision,
    LinkPredRecall,
    LinkPredNDCG,
)
from torch_geometric.nn import GCNConv, to_hetero

import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from torch_geometric.data import HeteroData
from torch_geometric.transforms import ToUndirected

import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from torch_geometric.data import HeteroData
from torch_geometric.nn import SAGEConv, to_hetero
import torch.nn.functional as F
import numpy as np
from sklearn.model_selection import train_test_split
import random
import os

# set env

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
g=torch.Generator()
g.manual_seed(seed)
# Ensure deterministic behavior (may slow down performance slightly)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms = True
CUBLAS_WORKSPACE_CONFIG=':16:8'
os.environ['CUBLAS_WORKSPACE_CONFIG'] = CUBLAS_WORKSPACE_CONFIG
os.environ['PYTHONHASHSEED'] = str(seed)
# Set a random seed for reproducibility
def set_seed():
  seed = 0
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  np.random.seed(seed)
  random.seed(seed)
  g=torch.Generator()
  g.manual_seed(seed)
  # Ensure deterministic behavior (may slow down performance slightly)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  torch.use_deterministic_algorithms = True
  CUBLAS_WORKSPACE_CONFIG=':16:8'
  os.environ['CUBLAS_WORKSPACE_CONFIG'] = CUBLAS_WORKSPACE_CONFIG
  os.environ['PYTHONHASHSEED'] = str(seed)
set_seed()

In [8]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# cosine search

this is the definiction of the search function that is similar to MIPS (maximum inner product search) but with cosine similarity instead of dot product.

In [9]:
import torch
import warnings
from typing import NamedTuple, Optional
from torch import Tensor
import torch.nn.functional as F

class KNNOutput(NamedTuple):
    score: Tensor
    index: Tensor

class KNNIndex:
    def __init__(
        self,
        emb: Optional[Tensor] = None,
        reserve: Optional[int] = None,
    ):
        warnings.filterwarnings('ignore', '.*TypedStorage is deprecated.*')

        self.index: Optional[Tensor] = None
        self.reserve = reserve

        if emb is not None:
            self.add(emb)

    @property
    def numel(self) -> int:
        if self.index is None:
            return 0
        return self.index.size(0)

    def add(self, emb: Tensor):
        if self.index is None:
            self.index = emb.detach().clone()
        else:
            self.index = torch.cat([self.index, emb.detach()], dim=0)

    def search(
        self,
        emb: Tensor,
        k: int,
        exclude_links: Optional[Tensor] = None,
    ) -> KNNOutput:
        if self.index is None:
            raise RuntimeError(f"'{self.__class__.__name__}' is not yet "
                               "initialized. Please call `add(...)` first.")

        if emb.dim() != 2:
            raise ValueError(f"'emb' needs to be two-dimensional "
                             f"(got {emb.dim()} dimensions)")



        scores = torch.cosine_similarity(emb.unsqueeze(1), self.index.unsqueeze(0), dim=-1)
        scores, indices = torch.topk(scores, k=k, dim=1, largest=True, sorted=True)

        return KNNOutput(scores, indices)

class ApproxCosineKNNIndex(KNNIndex):
    def __init__(
        self,
        emb: Optional[Tensor] = None,
        reserve: Optional[int] = None,
    ):
        super().__init__(emb=emb, reserve=reserve)


# model training

In [10]:
#set k value
k=10

## model definition

In [11]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_channels):
        super().__init__()
        self.conv1 = SAGEConv(input_dim, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

class CosineSimilarityDecoder(torch.nn.Module):
    def forward(self, x_dict, edge_label_index):
        x_src = x_dict['user'][edge_label_index[0]]
        x_dst = x_dict['job'][edge_label_index[1]]
        return torch.cosine_similarity(x_src, x_dst, dim=1)


class Model(torch.nn.Module):
    def __init__(self, input_dim, hidden_channels):
        super().__init__()
        self.encoder = GCN(input_dim, hidden_channels)
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr='sum')
        self.decoder = CosineSimilarityDecoder()

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        x_dict = self.encoder(x_dict, edge_index_dict)
        cosine_similarity = self.decoder(x_dict, edge_label_index)
        return cosine_similarity



## training

In [12]:
def train():
    model.train()

    total_loss = total_examples = 0
    for batch in tqdm(train_loader):
        batch = batch.to(device)
        optimizer.zero_grad()

        out = model(
            batch.x_dict,
            batch.edge_index_dict,
            batch['user', 'job'].edge_label_index,
        )
        y = batch['user', 'job'].edge_label

        loss = F.binary_cross_entropy_with_logits(out, y)
        loss.backward()
        optimizer.step()

        total_loss += float(loss) * y.numel()
        total_examples += y.numel()

    return total_loss / total_examples



## test

In [13]:
@torch.no_grad()
def test(edge_label_index, exclude_links):
    model.eval()

    dst_embs = []
    for batch in dst_loader:
        batch = batch.to(device)
        emb = model.encoder(batch.x_dict, batch.edge_index_dict)['job']
        emb = emb[:batch['job'].batch_size]
        dst_embs.append(emb)
    dst_emb = torch.cat(dst_embs, dim=0)

    index = ApproxCosineKNNIndex(dst_emb)

    ndcg_metric = LinkPredNDCG(k=k).to(device)
    precision_metric = LinkPredPrecision(k=k).to(device)
    recall_metric = LinkPredRecall(k=k).to(device)

    true_values = []
    predicted_values = []

    num_processed = 0
    for batch in src_loader:
        batch = batch.to(device)

        emb = model.encoder(batch.x_dict, batch.edge_index_dict)['user']
        emb = emb[:batch['user'].batch_size]

        _edge_label_index = edge_label_index.sparse_narrow(
            dim=0,
            start=num_processed,
            length=emb.size(0),
        )
        _exclude_links = exclude_links.sparse_narrow(
            dim=0,
            start=num_processed,
            length=emb.size(0),
        )
        num_processed += emb.size(0)

        output = index.search(emb, dst_emb.size(0), _exclude_links)

        ndcg_metric.update(output.index[:, :k], _edge_label_index)
        precision_metric.update(output.index[:, :k], _edge_label_index)
        recall_metric.update(output.index[:, :k], _edge_label_index)

        true_values.append(_edge_label_index.to_dense().cpu())
        predicted_values.append(output.score.cpu())

    true_values = torch.cat(true_values, dim=0)
    predicted_values = torch.cat(predicted_values, dim=0)

    mse = F.mse_loss(predicted_values, true_values)
    rmse = torch.sqrt(mse)

    mae = F.l1_loss(predicted_values, true_values)

    ndcg = float(ndcg_metric.compute())
    precision = float(precision_metric.compute())
    recall = float(recall_metric.compute())

    return (
        ndcg,
        precision,
        recall,
        rmse.item(),
        mae.item(),
    )



# experiment

In [ ]:
# Initialize an empty list to store the results
results_list = []

## data without LDA

In [14]:
data = torch.load('/content/drive/MyDrive/data/graph/graph_gptl_sansLDA.pt', map_location=torch.device('cpu'))

# Get the number of edges in your graph data
num_edges = data['user', 'applies', 'job'].edge_index.size(1)

# Define the ratio for splitting
train_ratio = 0.8

# Generate train and test indices using train_test_split
train_indices, test_indices = train_test_split(range(num_edges), train_size=train_ratio, random_state=42)

# Convert indices to tensors
train_index = torch.tensor(train_indices)
test_index = torch.tensor(test_indices)

# Set up loaders using train and test indices
kwargs = dict(
    data=data,
    num_neighbors=[-1, -1],
    batch_size=32,
    num_workers=0,
    persistent_workers=False,
    shuffle=False,
    worker_init_fn=seed_worker,  # to ensure reproducibility
    generator=g,
)

train_loader = LinkNeighborLoader(
    edge_label_index=(('user', 'applies', 'job'), data['user', 'applies', 'job'].edge_index[:, train_index]),
    neg_sampling=dict(mode='binary', amount=2),
    **kwargs,
)

# During testing, we sample node-level subgraphs from both endpoints to
# retrieve their embeddings.
# This allows us to do efficient k-NN search on top of embeddings:
src_loader = NeighborLoader(
    input_nodes='user',
    **kwargs,
)
dst_loader = NeighborLoader(
    input_nodes='job',
    **kwargs,
)

# Save test edges and the edges we want to exclude when evaluating:
sparse_size = (data['user'].num_nodes, data['job'].num_nodes)
test_edge_label_index = EdgeIndex(
    data['user', 'applies', 'job'].edge_index[:, test_index].to(device),
    sparse_size=sparse_size,
).sort_by('row')[0]
test_exclude_links = EdgeIndex(
    data['user', 'applies', 'job'].edge_index[:, train_index].to(device),
    sparse_size=sparse_size,
).sort_by('row')[0]

In [ ]:
# Define the fixed hidden channel size
hidden_channels = 32
set_seed()
model = Model(input_dim=768, hidden_channels=hidden_channels).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 51):
        set_seed()
        # Train the model for epochs and calculate the training loss
        train_loss = train()
        print(f'Epoch: {epoch:02d}, Loss: {train_loss:.4f}\n')

torch.save(model, '/content/drive/MyDrive/models/model_sansLDA_stable.pt')
# Test the model and calculate evaluation metrics after training for all epochs
val_ndcg, val_precision, val_recall, rmse , mae = test(test_edge_label_index, test_exclude_links)
print(f'Test NDCG@10: {val_ndcg:.4f}, Test Precision@10: {val_precision:.4f}, Test Recall@10: {val_recall:.4f}')

# Append the results to the list with the model name and hidden channel size included
results_list.append({
        'data': 'sans_LDA',
        'NDCG@10': val_ndcg,
        'Precision@10': val_precision,
        'Recall@10': val_recall,
        'RMSE': rmse,
        'MAE': mae,
})


100%|██████████| 25/25 [00:00<00:00, 33.34it/s]


Epoch: 01, Loss: 0.5954



100%|██████████| 25/25 [00:00<00:00, 36.59it/s]


Epoch: 02, Loss: 0.5757



100%|██████████| 25/25 [00:01<00:00, 16.74it/s]


...

Epoch: 48, Loss: 0.4664



100%|██████████| 25/25 [00:00<00:00, 61.72it/s]


Epoch: 49, Loss: 0.4666



100%|██████████| 25/25 [00:00<00:00, 58.09it/s]


Epoch: 50, Loss: 0.4663

Test NDCG@10: 0.0959, Test Precision@10: 0.0224, Test Recall@10: 0.1899


## data without user-user

In [ ]:
data = torch.load('/content/drive/MyDrive/data/graph/graph_gptl_sansUserS.pt', map_location=torch.device('cpu'))

# Get the number of edges in your graph data
num_edges = data['user', 'applies', 'job'].edge_index.size(1)

# Define the ratio for splitting
train_ratio = 0.8

# Generate train and test indices using train_test_split
train_indices, test_indices = train_test_split(range(num_edges), train_size=train_ratio, random_state=42)

# Convert indices to tensors
train_index = torch.tensor(train_indices)
test_index = torch.tensor(test_indices)

# Set up loaders using train and test indices
kwargs = dict(
    data=data,
    num_neighbors=[-1, -1],
    batch_size=32,
    num_workers=0,
    persistent_workers=False,
    shuffle=False,
    worker_init_fn=seed_worker,  # to ensure reproducibility
    generator=g,
)

train_loader = LinkNeighborLoader(
    edge_label_index=(('user', 'applies', 'job'), data['user', 'applies', 'job'].edge_index[:, train_index]),
    neg_sampling=dict(mode='binary', amount=2),
    **kwargs,
)

# During testing, we sample node-level subgraphs from both endpoints to
# retrieve their embeddings.
# This allows us to do efficient k-NN search on top of embeddings:
src_loader = NeighborLoader(
    input_nodes='user',
    **kwargs,
)
dst_loader = NeighborLoader(
    input_nodes='job',
    **kwargs,
)

# Save test edges and the edges we want to exclude when evaluating:
sparse_size = (data['user'].num_nodes, data['job'].num_nodes)
test_edge_label_index = EdgeIndex(
    data['user', 'applies', 'job'].edge_index[:, test_index].to(device),
    sparse_size=sparse_size,
).sort_by('row')[0]
test_exclude_links = EdgeIndex(
    data['user', 'applies', 'job'].edge_index[:, train_index].to(device),
    sparse_size=sparse_size,
).sort_by('row')[0]

In [ ]:
# Define the fixed hidden channel size
hidden_channels = 32
set_seed()
model = Model(input_dim=768, hidden_channels=hidden_channels).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 51):
        set_seed()
        # Train the model for epochs and calculate the training loss
        train_loss = train()
        print(f'Epoch: {epoch:02d}, Loss: {train_loss:.4f}\n')
torch.save(model, '/content/drive/MyDrive/models/model_sansUserS_stable.pt')
# Test the model and calculate evaluation metrics after training for all epochs
val_ndcg, val_precision, val_recall, rmse , mae = test(test_edge_label_index, test_exclude_links)
print(f'Test NDCG@10: {val_ndcg:.4f}, Test Precision@10: {val_precision:.4f}, Test Recall@10: {val_recall:.4f}')

# Append the results to the list with the model name and hidden channel size included
results_list.append({
        'data': 'sans_UserS',
        'NDCG@10': val_ndcg,
        'Precision@10': val_precision,
        'Recall@10': val_recall,
        'RMSE': rmse,
        'MAE': mae,
})

100%|██████████| 25/25 [00:03<00:00,  6.69it/s]


Epoch: 01, Loss: 0.6044



100%|██████████| 25/25 [00:03<00:00,  6.63it/s]


Epoch: 02, Loss: 0.5780



100%|██████████| 25/25 [00:04<00:00,  5.50it/s]


...

Epoch: 48, Loss: 0.4650



100%|██████████| 25/25 [00:03<00:00,  6.30it/s]


Epoch: 49, Loss: 0.4663



100%|██████████| 25/25 [00:05<00:00,  4.87it/s]


Epoch: 50, Loss: 0.4646

Test NDCG@10: 0.1223, Test Precision@10: 0.0279, Test Recall@10: 0.2384


## data without job-job

In [ ]:
data = torch.load('/content/drive/MyDrive/data/graph/graph_gptl_sansUserS.pt', map_location=torch.device('cpu'))

# Get the number of edges in your graph data
num_edges = data['user', 'applies', 'job'].edge_index.size(1)

# Define the ratio for splitting
train_ratio = 0.8

# Generate train and test indices using train_test_split
train_indices, test_indices = train_test_split(range(num_edges), train_size=train_ratio, random_state=42)

# Convert indices to tensors
train_index = torch.tensor(train_indices)
test_index = torch.tensor(test_indices)

# Set up loaders using train and test indices
kwargs = dict(
    data=data,
    num_neighbors=[-1, -1],
    batch_size=32,
    num_workers=0,
    persistent_workers=False,
    shuffle=False,
    worker_init_fn=seed_worker,  # to ensure reproducibility
    generator=g,
)

train_loader = LinkNeighborLoader(
    edge_label_index=(('user', 'applies', 'job'), data['user', 'applies', 'job'].edge_index[:, train_index]),
    neg_sampling=dict(mode='binary', amount=2),
    **kwargs,
)

# During testing, we sample node-level subgraphs from both endpoints to
# retrieve their embeddings.
# This allows us to do efficient k-NN search on top of embeddings:
src_loader = NeighborLoader(
    input_nodes='user',
    **kwargs,
)
dst_loader = NeighborLoader(
    input_nodes='job',
    **kwargs,
)

# Save test edges and the edges we want to exclude when evaluating:
sparse_size = (data['user'].num_nodes, data['job'].num_nodes)
test_edge_label_index = EdgeIndex(
    data['user', 'applies', 'job'].edge_index[:, test_index].to(device),
    sparse_size=sparse_size,
).sort_by('row')[0]
test_exclude_links = EdgeIndex(
    data['user', 'applies', 'job'].edge_index[:, train_index].to(device),
    sparse_size=sparse_size,
).sort_by('row')[0]

In [ ]:
# Define the fixed hidden channel size
hidden_channels = 32
set_seed()
model = Model(input_dim=768, hidden_channels=hidden_channels).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 51):
        set_seed()
        # Train the model for epochs and calculate the training loss
        train_loss = train()
        print(f'Epoch: {epoch:02d}, Loss: {train_loss:.4f}\n')
torch.save(model, '/content/drive/MyDrive/models/model_sansJobS_stable.pt')
# Test the model and calculate evaluation metrics after training for all epochs
val_ndcg, val_precision, val_recall , rmse , mae= test(test_edge_label_index, test_exclude_links)
print(f'Test NDCG@10: {val_ndcg:.4f}, Test Precision@10: {val_precision:.4f}, Test Recall@10: {val_recall:.4f}')

# Append the results to the list with the model name and hidden channel size included
results_list.append({
        'data': 'sans_JobS',
        'NDCG@10': val_ndcg,
        'Precision@10': val_precision,
        'Recall@10': val_recall,
        'RMSE': rmse,
        'MAE': mae,
})

100%|██████████| 25/25 [00:03<00:00,  7.02it/s]


Epoch: 01, Loss: 0.6044



100%|██████████| 25/25 [00:03<00:00,  6.70it/s]


Epoch: 02, Loss: 0.5780



100%|██████████| 25/25 [00:05<00:00,  4.90it/s]


...

Epoch: 48, Loss: 0.4650



100%|██████████| 25/25 [00:03<00:00,  6.61it/s]


Epoch: 49, Loss: 0.4663



100%|██████████| 25/25 [00:04<00:00,  6.16it/s]


Epoch: 50, Loss: 0.4646

Test NDCG@10: 0.1223, Test Precision@10: 0.0279, Test Recall@10: 0.2384


# results

In [ ]:
result_LDA = pd.DataFrame(results_list)
result_LDA.to_csv('/content/drive/MyDrive/data/results_LDA_stable.csv')

In [ ]:
result_LDA

,data,NDCG@10,Precision@10,Recall@10,RMSE,MAE
0,sans_LDA,0.095886,0.022424,0.189899,0.636101,0.568295
1,sans_UserS,0.122280,0.027879,0.238384,0.588152,0.517620
2,sans_JobS,0.122280,0.027879,0.238384,0.588152,0.517620


In [ ]:
res=result_LDA
res=res.sort_values(by=['Precision@10', 'Recall@10','NDCG@10','RMSE','MAE'], ascending=False)
print(res.iloc[:4,:])

         data   NDCG@10  Precision@10  Recall@10      RMSE       MAE
1  sans_UserS  0.122280      0.027879   0.238384  0.588152  0.517620
2   sans_JobS  0.122280      0.027879   0.238384  0.588152  0.517620
0    sans_LDA  0.095886      0.022424   0.189899  0.636101  0.568295
